## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [3]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import requests
import os
from typing import Dict
from IPython.display import display, Markdown

In [4]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use free and low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern. Also student Christian W. pointed out that OpenAI can sometimes charge for multiple searches for a single call, so it could sometimes cost more than 2.5 cents per call.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [5]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [6]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems. LangChain stands out as a modular framework for building applications powered by large language models (LLMs). It provides tools for chaining prompts, models, memory, and external tools, facilitating the creation of complex workflows. LangGraph extends LangChain by introducing a graph-based approach to define agent workflows, offering explicit control over information flow and enabling branching and debugging of complex behaviors. CrewAI focuses on role-based agent collaboration, allowing the creation of specialized agents that work together on complex projects, akin to a team environment. AutoGen, developed by Microsoft, specializes in orchestrating multiple AI agents to form autonomous, event-driven systems capable of handling complex, multi-agent tasks seamlessly. Eliza is an open-source, Web3-friendly agentic framework that integrates seamlessly with blockchain applications, enabling the deployment of decentralized AI agents. Agent Lightning is a flexible and extensible framework that enables reinforcement learning-based training of LLMs for any AI agent, allowing seamless integration with existing agents developed via diverse methods. AutoAgent is a fully-automated, zero-code framework that enables users to create and deploy LLM agents through natural language alone, serving as a versatile multi-agent system for general AI assistants. These frameworks represent the forefront of AI agent development, each contributing to the advancement of intelligent, autonomous systems in 2025. 

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [7]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [8]:

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To gather information on the latest AI agent frameworks currently available or being developed.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To find comparisons and reviews of emerging AI frameworks in 2025.', query='best AI frameworks for agents 2025'), WebSearchItem(reason='To explore advancements and innovations in AI agent technology expected or realized in 2025.', query='AI agent technology trends 2025')]


In [9]:
@function_tool
def send_research_notification(report_content: str) -> Dict[str, str]:
    """ Send a research report as a Pushover notification """
    url = "https://api.pushover.net/1/messages.json"
    
    # Clean up markdown for better readability in notifications
    import re
    clean_content = re.sub(r'#{1,6}\s*', '', report_content)  # Remove markdown headers
    clean_content = re.sub(r'\*\*(.*?)\*\*', r'\1', clean_content)  # Remove bold markdown
    clean_content = re.sub(r'\*(.*?)\*', r'\1', clean_content)  # Remove italic markdown
    clean_content = re.sub(r'\n\n+', '\n\n', clean_content)  # Clean multiple newlines
    
    # Truncate if too long for Pushover (max 1024 chars)
    if len(clean_content) > 900:
        clean_content = clean_content[:900] + "...\n\n[Report truncated - see full version in trace]"
    
    data = {
        "token": os.environ.get('PUSHOVER_TOKEN'),
        "user": os.environ.get('PUSHOVER_USER'),
        "message": clean_content,
        "title": "Research Report Complete"
    }
    
    response = requests.post(url, data=data)
    if response.status_code == 200:
        return {"status": "success", "message": "Research report notification sent successfully"}
    else:
        return {"status": "error", "message": f"Error {response.status_code}: {response.text}"}

In [10]:
send_research_notification

FunctionTool(name='send_research_notification', description='Send a research report as a Pushover notification', params_json_schema={'properties': {'report_content': {'title': 'Report Content', 'type': 'string'}}, 'required': ['report_content'], 'title': 'send_research_notification_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10bb1bb00>, strict_json_schema=True, is_enabled=True)

In [11]:
INSTRUCTIONS = """You are able to send a research report notification via Pushover.
You will be provided with a detailed markdown report. You should use your tool to send one notification 
with the report content. The notification should be clean and readable."""

notification_agent = Agent(
    name="Notification agent",
    instructions=INSTRUCTIONS,
    tools=[send_research_notification],
    model="gpt-4o-mini",
)



In [12]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [13]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and send notification

In [14]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_notification(report: ReportData):
    """ Use the notification agent to send a notification with the report """
    print("Sending notification...")
    result = await Runner.run(notification_agent, report.markdown_report)
    print("Notification sent")
    return report

### Showtime!

In [15]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_notification(report)  
    print("Hooray!")




Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Sending notification...
Notification sent
Hooray!


### As always, take a look at the trace

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Congratulations on your progress, and a request</h2>
            <span style="color:#00cc00;">You've reached an important moment with the course; you've created a valuable Agent using one of the latest Agent frameworks. You've upskilled, and unlocked new commercial possibilities. Take a moment to celebrate your success!<br/><br/>Something I should ask you -- my editor would smack me if I didn't mention this. If you're able to rate the course on Udemy, I'd be seriously grateful: it's the most important way that Udemy decides whether to show the course to others and it makes a massive difference.<br/><br/>And another reminder to <a href="https://www.linkedin.com/in/eddonner/">connect with me on LinkedIn</a> if you wish! If you wanted to post about your progress on the course, please tag me and I'll weigh in to increase your exposure.
            </span>
        </td>
    </tr>